
# MLflow プロンプト最適化

https://docs.databricks.com/aws/ja/mlflow3/genai/prompt-version-mgmt/prompt-registry/automatically-optimize-prompts

> # プロンプト最適化（実験的機能）
> MLflowでは、```mlflow.genai.optimize_prompt()```APIを使用したMLflowの統合インターフェースを通じて、プロンプトを高度なプロンプト最適化手法に組み込むことができます。この機能は、評価指標とラベル付きデータを活用して、プロンプトを自動的に改善するのに役立ちます。現在、このAPIはDSPyのMIPROv2アルゴリズムをサポートしています。
> 
> ## 主なメリット
> - **統合インターフェース:** 中立的なインターフェースを介して最先端のプロンプト最適化アルゴリズムにアクセスできます。
> - **プロンプト管理:** MLflow プロンプト レジストリと統合して、再利用性、バージョン管理、系統を実現します。
> - **評価:** MLflow の評価機能を使用してプロンプトのパフォーマンスを総合的に評価します。
> 

## 簡易チュートリアル

In [0]:
%pip install -U "mlflow[databricks]>=3.1.0" databricks-langchain langgraph dspy databricks-agents

%restart_python

In [0]:
import mlflow
from mlflow.entities import Prompt
mlflow.set_registry_uri("databricks-uc")

CATALOG = "workspace"
SCHEMA = "default"

In [0]:
# First prompt for summarization.
qa_prompt = mlflow.genai.register_prompt(
    name=f"{CATALOG}.{SCHEMA}.qa_prompt",
    template="次の質問に対して日本語で回答してください:{{question}}",
)

qa_prompt

In [0]:
import pandas as pd

# 質問と回答のペアをリストとして定義
train_data = [
    {
        "inputs": {"question": "Databricksとは何ですか？"},
        "expectations": {
            "answer": "Databricksは、データエンジニアリング、データサイエンス、機械学習のための統合データ分析プラットフォームです。"
        },
    },
    {
        "inputs": {"question": "Databricksの主な機能は何ですか？"},
        "expectations": {
            "answer": "Databricksの主な機能には、データの統合、分析、機械学習モデルのトレーニングとデプロイがあります。"
        },
    },
    {
        "inputs": {"question": "Databricksで使用できるプログラミング言語は何ですか？"},
        "expectations": {
            "answer": "Databricksでは、Python、SQL、R、Scalaなどのプログラミング言語を使用できます。"
        },
    },
    {
        "inputs": {"question": "Databricksのノートブックとは何ですか？"},
        "expectations": {
            "answer": "Databricksのノートブックは、データ分析や機械学習のコードを記述、実行、共有するためのインタラクティブな環境です。"
        },
    },
    {
        "inputs": {"question": "Databricksのクラスターとは何ですか？"},
        "expectations": {
            "answer": "Databricksのクラスターは、データ処理や分析のために使用されるコンピューティングリソースの集合です。"
        },
    },
]
eval_data = [
    {
        "inputs": {"question": "DatabricksのDelta Lakeとは何ですか？"},
        "expectations": {
            "answer": "Delta Lakeは、Databricks上で提供される信頼性の高いデータレイクソリューションで、ACIDトランザクションやスキーマエンフォースメントをサポートします。"
        },
    },
    {
        "inputs": {"question": "DatabricksのMLflowとは何ですか？"},
        "expectations": {"answer": "MLflowは、機械学習モデルのライフサイクル管理を支援するオープンソースプラットフォームです。"},
    },
    {
        "inputs": {"question": "Databricksのジョブとは何ですか？"},
        "expectations": {
            "answer": "Databricksのジョブは、スケジュールされたデータ処理タスクやワークフローを自動化するための機能です。"
        },
    },
    {
        "inputs": {"question": "Databricksのワークスペースとは何ですか？"},
        "expectations": {
            "answer": "Databricksのワークスペースは、データ分析や機械学習プロジェクトを管理するためのコラボレーション環境です。"
        },
    },
    {
        "inputs": {"question": "DatabricksのUnity Catalogとは何ですか？"},
        "expectations": {
            "answer": "Unity Catalogは、Databricks上でデータガバナンスとセキュリティを提供するための統合データカタログです。"
        },
    },
]

# pandasデータフレームに変換
pdf = pd.DataFrame(train_data)

# データフレームを表示
display(pdf)

In [0]:
from typing import Any
from mlflow.genai.scorers import Correctness
from mlflow.genai.optimize import OptimizerConfig, LLMParams
from mlflow.genai.scorers import scorer
import os

# OpenAI Clientが利用できるように、現在のCredentialをOPENAI_API_KEYに登録
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
os.environ["OPENAI_API_KEY"] = mlflow_creds.token

# Correctnessスコアを計算するbuilt-inオブジェクトを作成
_correctness = Correctness()

# プロンプト最適化のための評価関数（確からしさのテスト)
@scorer
def correctness(inputs, outputs, expectations):
    expectations = {"expected_response": expectations.get("answer")}
    return (
        _correctness(inputs=inputs, outputs=outputs, expectations=expectations).value
        == "yes"
    )

# 最適化対象のプロンプト
prompt = mlflow.genai.load_prompt(f"prompts:/{CATALOG}.{SCHEMA}.qa_prompt/1")

# プロンプトを最適化
result = mlflow.genai.optimize_prompt(
    target_llm_params=LLMParams(
        model_name="openai/databricks-llama-4-maverick",
        base_uri=f"{mlflow_creds.host}/serving-endpoints",
    ),
    prompt=prompt,
    train_data=train_data,
    eval_data=eval_data,
    scorers=[correctness],
    optimizer_config=OptimizerConfig(
        num_instruction_candidates=8,
        max_few_show_examples=2,
        # verbose=True,
        autolog=True,
    ),
)

# 最適化結果のプロンプトレジストリのURLを表示
print(result.prompt.uri)


In [0]:
from pprint import pprint
prompt = mlflow.genai.load_prompt(result.prompt.uri)

print(prompt.template)

In [0]:
import mlflow
from openai import OpenAI
import codecs

# MLflowの自動ロギングを有効にして、アプリケーションにトレースを追加
mlflow.openai.autolog()

# 実行ノートブックと同じ資格情報を使用してOpenAIクライアント経由でDatabricks LLMに接続
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
)

# レジストリからプロンプトをロード
first_prompt = mlflow.genai.load_prompt(f"prompts:/{CATALOG}.{SCHEMA}.qa_prompt/1")
optimized_prompt = mlflow.genai.load_prompt(result.prompt.uri)
endpoint = "databricks-llama-4-maverick"

def predict(prompt):
    formatted_prompt = prompt.format(question="Databricksとは何ですか?")
    
    # LLMを呼び出す
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {
                "role": "user",
                "content": formatted_prompt,
            },
        ],
    )
    return response.choices[0].message.content


for p in [first_prompt, optimized_prompt]:
    print(predict(p))

## 公式Notebook

https://docs.databricks.com/aws/ja/notebooks/source/mlflow/prompt-optimization.html

In [0]:
%pip install -U "mlflow[databricks]>=3.1.0" langchain-community langchain-openai beautifulsoup4 langgraph dspy databricks-agents

%restart_python

In [0]:
# TODO: If necessary, change the catalog and schema name here
CATALOG = "workspace"
SCHEMA = "default"

In [0]:
import mlflow
from mlflow.entities import Prompt
mlflow.set_registry_uri("databricks-uc")

In [0]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

split_docs = text_splitter.split_documents(docs)
print(f"Generated {len(split_docs)} documents.")

In [0]:
from langchain.chat_models import init_chat_model

mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
llm = init_chat_model(
    # "databricks-llama-4-maverick",
    "databricks-meta-llama-3-1-405b-instruct",
    model_provider="openai",
    api_key=mlflow_creds.token,
    base_url=f"{mlflow_creds.host}/serving-endpoints",
)


In [0]:
# First prompt for summarization.
summary_prompt = mlflow.genai.register_prompt(
    name=f"{CATALOG}.{SCHEMA}.summary_prompt",
    template="Write a concise summary of the following:{{content}}",
)

In [0]:
summary_prompt

In [0]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

summary_chain = llm | StrOutputParser()

@mlflow.trace()
def call_summary_chain(content):
  return summary_chain.invoke([HumanMessage(summary_prompt.format(content=content))])

In [0]:
# Second prompt for topic extraction.
topic_prompt = mlflow.genai.register_prompt(name=f"{CATALOG}.{SCHEMA}.topic_prompt",
                       template="""
The following is the summary:
{{summary}}
Extract the main topic in a few words.
Return the response in JSON format: {"topic": "..."}
""")

topic_chain = llm | JsonOutputParser()

@mlflow.trace()
def call_topic_chain(summary):
  return topic_chain.invoke([HumanMessage(topic_prompt.format(summary=summary))])

In [0]:
from langchain_core.messages import HumanMessage, SystemMessage

@mlflow.trace
def agent(content):
  summary = call_summary_chain(content=content)
  return call_topic_chain(summary=summary)["topic"]

In [0]:
# Enable Autologging
mlflow.langchain.autolog()

In [0]:
# Run the agent
for doc in split_docs:
  try:
    print(agent(doc.page_content))
  except Exception as e:
    print(e)
    pass

## Dataset Creation

In [0]:
import mlflow

# Extract the inputs and outputs of the second LLM call
traces = mlflow.search_traces(extract_fields=[
  "call_topic_chain.inputs",
  "call_topic_chain.outputs",
])

In [0]:
traces.head(10)

In [0]:
from mlflow.genai import datasets

EVAL_DATASET_NAME=f"{CATALOG}.{SCHEMA}.data"
dataset = datasets.create_dataset(EVAL_DATASET_NAME)

In [0]:
dataset

In [0]:
# Create a dataset by treating the agent outputs as the default expectations.
traces = traces.rename(
    columns={
      "call_topic_chain.inputs": "inputs",
      "call_topic_chain.outputs": "expectations",
    }
)[["inputs", "expectations"]]
traces = traces.dropna()
dataset.merge_records(traces)


## Labeling

In [0]:
dataset = datasets.get_dataset(EVAL_DATASET_NAME)
dataset.merge_records([])

In [0]:
dataset = dataset.to_df()
dataset.head()

## Optimize

In [0]:
import os
import mlflow
from typing import Any
from mlflow.genai.scorers import scorer
from mlflow.genai.optimize import OptimizerConfig, LLMParams

mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
os.environ["OPENAI_API_KEY"] = mlflow_creds.token


@scorer
def exact_match(expectations: dict[str, Any], outputs: dict[str, Any]) -> bool:
    return expectations == outputs

prompt = mlflow.genai.register_prompt(
    name=f"{CATALOG}.{SCHEMA}.qa",
    template="Answer the following question: {{question}}",
)

result = mlflow.genai.optimize_prompt(
    target_llm_params=LLMParams(
        model_name="openai/databricks-meta-llama-3-1-405b-instruct",
        base_uri=f"{mlflow_creds.host}/serving-endpoints",
    ),
    train_data=[
        {"inputs": {"question": f"{i}+1"}, "expectations": {"answer": f"{i + 1}"}}
        for i in range(100)
    ],
    scorers=[exact_match],
    prompt=prompt.uri,
    optimizer_config=OptimizerConfig(num_instruction_candidates=5),
)

print(result.prompt.template)

In [0]:
result.prompt

In [0]:
import os
from typing import Any
import mlflow
from mlflow.genai.scorers import Correctness
from mlflow.genai.optimize import OptimizerConfig, LLMParams
from mlflow.genai.scorers import scorer

_correctness = Correctness()


@scorer
def correctness(inputs, outputs, expectations):
    expectations = {"expected_response": expectations.get("topic")}
    return (
        _correctness(inputs=inputs, outputs=outputs, expectations=expectations).value
        == "yes"
    )


# Optimize the prompt
result = mlflow.genai.optimize_prompt(
    target_llm_params=LLMParams(
        model_name="openai/databricks-meta-llama-3-3-70b-instruct",
        base_uri=f"{mlflow_creds.host}/serving-endpoints",
    ),
    prompt=topic_prompt,
    train_data=dataset,
    scorers=[correctness],
    optimizer_config=OptimizerConfig(
        num_instruction_candidates=8,
        max_few_show_examples=2,
        verbose=True,
    ),
)

# The optimized prompt is automatically registered as a new version
# Open the prompt registry web site to check the new prompt
print(f"The new prompt URI: {result.prompt.uri}")